# NCTiles.jl examples

This notebook demonstrates the use of `NCTiles.jl` in combination with `MeshArrays.jl` (or in standalone mode) to create `NetCDF` files (or read them to memory). Several examples are included:

1. Write interpolated model output, on a regular `lat-lon` grid, to a single `NetCDF` file
  - 2D example
  - 3D example
2. Write tiled model output, on a tiled C-grid, to a `NCTiles` collection (multiple `NetCDF` files)
  - 2D, free surface example
  - 3D, temperature example
  - 3D, C-grid vector example

Running the examples requires that you first download and decompress test files:

```
git clone https://github.com/gaelforget/nctiles-testcases
gunzip nctiles-testcases/diags/*.gz
```

### Packages & input files

_These will be used throughout the notebook_

In [1]:
using NCTiles, MeshArrays

if !isdir("../inputs/nctiles-testcases")
    run(`git clone https://github.com/gaelforget/nctiles-testcases ../inputs/nctiles-testcases`)
end
#run(`gunzip nctiles-testcases/diags/trsp_3d_set1.0000000732.data.gz`)

Helper functions will be used avoid code duplication below:

In [2]:
include("nctiles_helper_functions.jl")

addvfgridvars

### Back-end and file paths

_These will be used throughout the notebook_

In [3]:
# Back-end
nc=NCTiles.NCDatasets

# Paths
datadir = "../inputs/nctiles-testcases/"
availdiagsfile = joinpath(datadir,"available_diagnostics.log")
readmefile = joinpath(datadir,"README")
griddir = joinpath(datadir,"grid_float32/")
nativedir = joinpath(datadir,"diags/")
interpdir = joinpath(datadir,"diags_interp/")

resultsdir = "../outputs/nctiles-newfiles/"
if ~ispath(resultsdir); mkpath(resultsdir); end

### Set up dimensions, sizes, and meta data

In [4]:
# Dimensions & sizes
prec = Float32
dep_l=-readbin(joinpath(griddir,"RF.data"),prec,(51,1))[2:end]
dep_c=-readbin(joinpath(griddir,"RC.data"),prec,(50,1))[:]
dep_lvar = NCvar("dep_l","m",size(dep_l),dep_l,Dict(["long_name" => "depth","positive"=>"down","standard_name"=>"depth"]),nc)
dep_cvar = NCvar("dep_c","m",size(dep_c),dep_c,Dict(["long_name" => "depth","positive"=>"down","standard_name"=>"depth"]),nc)
nsteps = 240
timeinterval = 3
time_steps = timeinterval/2:timeinterval:timeinterval*nsteps
time_units = "days since 1992-01-01 0:0:0"
timevar = NCvar("tim",time_units,Inf,time_steps,Dict(("long_name" => "time","standard_name" => "time")),nc)

# Meta data
README = readlines(readmefile)

3-element Array{String,1}:
 "Please replace this placeholder file with a descriptive"
 "paragraph that may provide a product name, version #,"  
 "and contact point for user support."                    

## Interpolated Data Examples

This example first interpolates 2D and 3D fields to a rectangular half-degree grid. See below for dimension and size definitions. The interpolated data is then written to a single `NetCDF` file for each variable.

In [5]:
lon_c=-179.75:0.5:179.75; lat_c=-89.75:0.5:89.75;
lon_cvar = NCvar("lon_c","degrees_east",size(lon_c),lon_c,Dict("long_name" => "longitude"),nc)
lat_cvar = NCvar("lat_c","degrees_north",size(lat_c),lat_c,Dict("long_name" => "longitude"),nc)
n1,n2,n3 = (length(lon_c),length(lat_c),length(dep_c))

(720, 360, 50)

### Interpolate

This section will take the original model output on the LLC90 grid and interpolate it to a rectangular half-degree grid. This will be done using `loop_task1` from [`CbiomesProcessing.jl`](https://github.com/gaelforget/CbiomesProcessing.jl). The following section is currently run using pre-interpolated data produced with [`gcmfaces`](https://github.com/gaelforget/gcmfaces) in Matlab.

### Output path

The following path will be used for this part's output.

In [6]:
writedir = joinpath(resultsdir,"interp")
if ~ispath(writedir); mkpath(writedir); end

### 2D example

Choose variable to process and get the corresponding list of input files

In [7]:
dataset = "state_2d_set1"
fldname = "ETAN"
flddatadir = joinpath(interpdir,fldname)
fnames = joinpath.(Ref(flddatadir),filter(x -> occursin(".data",x), readdir(flddatadir)))

3-element Array{String,1}:
 "../inputs/nctiles-testcases/diags_interp/ETAN/ETAN.0000000732.data"
 "../inputs/nctiles-testcases/diags_interp/ETAN/ETAN.0000001428.data"
 "../inputs/nctiles-testcases/diags_interp/ETAN/ETAN.0000002172.data"

Get meta data for the chosen variable

In [8]:
diaginfo = readAvailDiagnosticsLog(availdiagsfile,fldname)

Dict{String,Any} with 7 entries:
  "mate"    => ""
  "units"   => "m"
  "diagNum" => 23
  "fldname" => "ETAN"
  "title"   => "Surface Height Anomaly"
  "code"    => "SM      M1"
  "levs"    => 1

Define:

- a `BinData` struct to contain the file names, precision, and array size.
- a `NCvar` struct that sets up the subsequent `write` operation (incl. `BinData` struct.

In [9]:
flddata = BinData(fnames,prec,(n1,n2))
dims = [lon_cvar, lat_cvar, timevar]
field = NCvar(fldname,diaginfo["units"],dims,flddata,
    Dict("long_name" => diaginfo["title"]),nc)

NCvar("ETAN", "m", NCvar[NCvar("lon_c", "degrees_east", (720,), -179.75:0.5:179.75, Dict("long_name"=>"longitude"), NCDatasets), NCvar("lat_c", "degrees_north", (360,), -89.75:0.5:89.75, Dict("long_name"=>"longitude"), NCDatasets), NCvar("tim", "days since 1992-01-01 0:0:0", Inf, 1.5:3.0:718.5, Dict("long_name"=>"time","standard_name"=>"time"), NCDatasets)], BinData(["../inputs/nctiles-testcases/diags_interp/ETAN/ETAN.0000000732.data", "../inputs/nctiles-testcases/diags_interp/ETAN/ETAN.0000001428.data", "../inputs/nctiles-testcases/diags_interp/ETAN/ETAN.0000002172.data"], Float32, (720, 360), 1), Dict("long_name"=>"Surface Height Anomaly"), NCDatasets)

Create the NetCDF file and write data to it.

In [10]:
# Create the NetCDF file and populate with dimension and field info
ds,fldvar,dimlist = createfile(joinpath(writedir,fldname*".nc"),field,README)

# Add field and dimension data
addData(fldvar,field)
addDimData.(Ref(ds),field.dims)

# Close the file
close(ds)

### 3D example

In [11]:
# Get the filenames for our first dataset and other information about the field.
dataset = "WVELMASS"
fldname = "WVELMASS"
flddatadir = joinpath(interpdir,fldname)
fnames = flddatadir*'/'.*filter(x -> occursin(".data",x), readdir(flddatadir))
diaginfo = readAvailDiagnosticsLog(availdiagsfile,fldname)

# Define the field for writing using an NCvar struct. BinData contains the filenames
# where the data sits so it's only loaded when needed.
flddata = BinData(fnames,prec,(n1,n2,n3))
dims = [lon_cvar, lat_cvar, dep_lvar, timevar]
field = NCvar(fldname,diaginfo["units"],dims,flddata,Dict("long_name" => diaginfo["title"]),nc)

# Create the NetCDF file and populate with dimension and field info
ds,fldvar,dimlist = createfile(joinpath(writedir,fldname*".nc"),field,README)

# Add field and dimension data
addData(fldvar,field)
addDimData.(Ref(ds),field.dims)

# Close the file
close(ds)

## Tiled Data Examples

This example reads in global variables defined over a collection of subdomain arrays (_tiles_) using `MeshArrays.jl`, and writes them to a collection of `NetCDF` files (_nctiles_) using `NCTiles.jl`

### Output path

The following path will be used for this part's output.

In [12]:
writedir = joinpath(resultsdir,"tiled")
~ispath(writedir) ? mkpath(writedir) : nothing

### Set up dimensions, sizes, and meta data

In [13]:
# Read grid into MeshArrays
mygrid = GridSpec("LatLonCap",griddir)
mygrid = gcmgrid(griddir,mygrid.class,mygrid.nFaces,
    mygrid.fSize, mygrid.ioSize, Float32, mygrid.read, mygrid.write)
gridvars = addvfgridvars(GridLoad(mygrid))

tilesize = (30,30)
(n1,n2,n3) = (90,1170,50)

# First two dimensions
icvar = NCvar("i_c","1",tilesize[1],1:tilesize[1],Dict("long_name" => "Cartesian coordinate 1"),nc)
jcvar = NCvar("j_c","1",tilesize[2],1:tilesize[2],Dict("long_name" => "Cartesian coordinate 2"),nc)
iwvar = NCvar("i_w","1",tilesize[1],1:tilesize[1],Dict("long_name" => "Cartesian coordinate 1"),nc)
jwvar = NCvar("j_w","1",tilesize[2],1:tilesize[2],Dict("long_name" => "Cartesian coordinate 2"),nc)
isvar = NCvar("i_s","1",tilesize[1],1:tilesize[1],Dict("long_name" => "Cartesian coordinate 1"),nc)
jsvar = NCvar("j_s","1",tilesize[2],1:tilesize[2],Dict("long_name" => "Cartesian coordinate 2"),nc)

# Land masks indicate which points are land, which are ocean
landC = gridvars["hFacC"]
landW = gridvars["hFacW"]
landS = gridvars["hFacS"]
for f in landC.fIndex
    for d in 1:size(landC,2)
        landC[f,d][landC[f,d].==0] .= NaN
        landC[f,d][landC[f,d].>0] .= 1

        landW[f,d][landW[f,d].==0] .= NaN
        landW[f,d][landW[f,d].>0] .= 1

        landS[f,d][landS[f,d].==0] .= NaN
        landS[f,d][landS[f,d].>0] .= 1
    end
end

# Variable indicating the depth/thickness of each cell
thicc = gridvars["DRF"]
thicl = gridvars["DRC"][2:end]

# TileData struct- calculates the locations of each tile in the
# data to retrieve when needed for writing
tilareaC = TileData(gridvars["RAC"],tilesize,mygrid)
tileinfo = tilareaC.tileinfo; numtiles = tilareaC.numtiles
tilareaW = TileData(gridvars["RAW"],tileinfo,tilesize,prec,numtiles)
tilareaS = TileData(gridvars["RAS"],tileinfo,tilesize,prec,numtiles)
tilland3D = TileData(landC,tileinfo,tilesize,prec,numtiles)
tilland2D = TileData(landC[:,1],tileinfo,tilesize,prec,numtiles)
tillandW = TileData(landW,tileinfo,tilesize,prec,numtiles)
tillandS = TileData(landS,tileinfo,tilesize,prec,numtiles)
tillatc = TileData(gridvars["YC"],tileinfo,tilesize,prec,numtiles)
tillonc = TileData(gridvars["XC"],tileinfo,tilesize,prec,numtiles)
tillatw = TileData(gridvars["YW"],tileinfo,tilesize,prec,numtiles)
tillonw = TileData(gridvars["XW"],tileinfo,tilesize,prec,numtiles)
tillats = TileData(gridvars["YS"],tileinfo,tilesize,prec,numtiles)
tillons = TileData(gridvars["XS"],tileinfo,tilesize,prec,numtiles)

# NCvar structs outline fields and their metadata to be written to the file
loncvar = NCvar("lon","degrees_east",[icvar,jcvar],tillonc,Dict("long_name" => "longitude"),nc)
latcvar = NCvar("lat","degrees_north",[icvar,jcvar],tillatc,Dict("long_name" => "latitude"),nc)
lonwvar = NCvar("lon","degrees_east",[iwvar,jwvar],tillonw,Dict("long_name" => "longitude"),nc)
latwvar = NCvar("lat","degrees_north",[iwvar,jwvar],tillatw,Dict("long_name" => "latitude"),nc)
lonsvar = NCvar("lon","degrees_east",[isvar,jsvar],tillons,Dict("long_name" => "longitude"),nc)
latsvar = NCvar("lat","degrees_north",[isvar,jsvar],tillats,Dict("long_name" => "latitude"),nc)
areacvar = NCvar("area","m^2",[icvar,jcvar],tilareaC,Dict(["long_name" => "grid cell area", "standard_name" => "cell_area"]),nc)
areawvar = NCvar("area","m^2",[iwvar,jwvar],tilareaW,Dict(["long_name" => "grid cell area", "standard_name" => "cell_area"]),nc)
areasvar = NCvar("area","m^2",[isvar,jsvar],tilareaS,Dict(["long_name" => "grid cell area", "standard_name" => "cell_area"]),nc)
land3Dvar = NCvar("land","1",[icvar,jcvar,dep_cvar],tilland3D,Dict(["long_name" => "land mask", "standard_name" => "land_binary_mask"]),nc)
land2Dvar = NCvar("land","1",[icvar,jcvar],tilland2D,Dict(["long_name" => "land mask", "standard_name" => "land_binary_mask"]),nc)
landwvar = NCvar("land","1",[iwvar,jwvar,dep_cvar],tillandW,Dict(["long_name" => "land mask", "standard_name" => "land_binary_mask"]),nc)
landsvar = NCvar("land","1",[isvar,jsvar,dep_cvar],tillandS,Dict(["long_name" => "land mask", "standard_name" => "land_binary_mask"]),nc)
thiccvar = NCvar("thic","m",dep_cvar,thicc,Dict("standard_name" => "cell_thickness"),nc)
thiclvar = NCvar("thic","m",dep_lvar,thicl,Dict("standard_name" => "cell_thickness"),nc)

NCvar("thic", "m", NCvar[NCvar("dep_l", "m", (50,), Float32[10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.01, 90.04, 100.15  …  2854.0, 3126.5, 3422.0, 3740.5, 4082.0, 4446.5, 4834.0, 5244.5, 5678.0, 6134.5], Dict("long_name"=>"depth","standard_name"=>"depth","positive"=>"down"), NCDatasets)], Float32[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.005, 10.02, 10.07, 10.215  …  261.0, 284.0, 307.0, 330.0, 353.0, 376.0, 399.0, 422.0, 445.0, 228.25], Dict("standard_name"=>"cell_thickness"), NCDatasets)

### 2D example

Choose variable to process and get the corresponding list of input files

In [14]:
dataset = "state_2d_set1"
fldname = "ETAN"
fnames = nativedir*'/'.*filter(x -> (occursin(".data",x) && occursin(dataset,x)), readdir(nativedir))
savepath = joinpath(writedir,fldname)
if ~ispath(savepath); mkpath(savepath); end
savenamebase = joinpath.(Ref(savepath),fldname)
diaginfo = readAvailDiagnosticsLog(availdiagsfile,fldname);

Prepare dictionary of `NCvar` structs and write to `NetCDF` files.

In [15]:
flddata = BinData(fnames,prec,(n1,n2))
tilfld = TileData(flddata,tilesize,mygrid)

dims = [icvar, jcvar, timevar]
coords = join(replace([dim.name for dim in dims],"i_c" => "lon", "j_c" => "lat")," ")
flds = Dict([fldname => NCvar(fldname,diaginfo["units"],dims,tilfld,Dict(["long_name" => diaginfo["title"], "coordinates" => coords]),nc),
            "lon" => loncvar,
            "lat" => latcvar,
            "area" => areacvar,
            "land" => land2Dvar
])

writeNetCDFtiles(flds,savenamebase,README)

### 3D example

In [16]:
# Get the filenames for our first dataset and other information about the field.
dataset = "state_3d_set1"
fldname = "THETA"
fnames = nativedir*'/'.*filter(x -> (occursin(".data",x) && occursin(dataset,x)), readdir(nativedir))
savepath = joinpath(writedir,fldname)
if ~ispath(savepath); mkpath(savepath); end
savenamebase = joinpath.(Ref(savepath),fldname)
diaginfo = readAvailDiagnosticsLog(availdiagsfile,fldname)

# Fields to be written to the file are indicated with a dictionary of NCvar structs.
flddata = BinData(fnames,prec,(n1,n2,n3))
dims = [icvar, jcvar, dep_cvar, timevar]
tilfld = TileData(flddata,tilesize,mygrid)
coords = join(replace([dim.name for dim in dims],"i_c" => "lon", "j_c" => "lat")," ")
flds = Dict([fldname => NCvar(fldname,diaginfo["units"],dims,tilfld,Dict(["long_name" => diaginfo["title"], "coordinates" => coords]),nc),
            "lon" => loncvar,
            "lat" => latcvar,
            "area" => areacvar,
            "land" => land3Dvar,
            "thic" => thiccvar
])

# Write to NetCDF files
writeNetCDFtiles(flds,savenamebase,README)

### 3D vector example

Here we process the three staggered components of a vector field (`UVELMASS`, `VVELMASS` and `WVELMASS`). On a `C-grid` these components are staggered in space.

First component : `UVELMASS`

In [17]:
# Get the filenames for our first dataset and create BinData struct
dataset = "trsp_3d_set1"
fldname = "UVELMASS"
fnames = nativedir*'/'.*filter(x -> (occursin(".data",x) && occursin(dataset,x)), readdir(nativedir))
savepath = joinpath(writedir,fldname)
if ~ispath(savepath); mkpath(savepath); end
savenamebase = joinpath.(Ref(savepath),fldname)
diaginfo = readAvailDiagnosticsLog(availdiagsfile,fldname)

# Define field- BinData contains the filenames where the data sits so it's only loaded when needed
flddata = BinData(fnames,prec,(n1,n2,n3))
dims = [iwvar, jwvar, dep_cvar, timevar]
tilfld = TileData(flddata,tilesize,mygrid)
coords = join(replace([dim.name for dim in dims],"i_w" => "lon", "j_w" => "lat")," ")
flds = Dict([fldname => NCvar(fldname,diaginfo["units"],dims,tilfld,Dict(["long_name" => diaginfo["title"], "coordinates" => coords]),nc),
            "lon" => lonwvar,
            "lat" => latwvar,
            "area" => areawvar,
            "land" => landwvar,
            "thic" => thiccvar
])

writeNetCDFtiles(flds,savenamebase,README)

Second component : `VVELMASS`

In [18]:
# Get the filenames for our first dataset and create BinData struct
dataset = "trsp_3d_set1"
fldname = "VVELMASS"
fnames = nativedir*'/'.*filter(x -> (occursin(".data",x) && occursin(dataset,x)), readdir(nativedir))
savepath = joinpath(writedir,fldname)
if ~ispath(savepath); mkpath(savepath); end
savenamebase = joinpath.(Ref(savepath),fldname)
diaginfo = readAvailDiagnosticsLog(availdiagsfile,fldname)

# Define field- BinData contains the filenames where the data sits so it's only loaded when needed
flddata = BinData(fnames,prec,(n1,n2,n3))
dims = [isvar, jsvar, dep_cvar, timevar]
tilfld = TileData(flddata,tilesize,mygrid)
coords = join(replace([dim.name for dim in dims],"i_s" => "lon", "j_s" => "lat")," ")
flds = Dict([fldname => NCvar(fldname,diaginfo["units"],dims,tilfld,Dict(["long_name" => diaginfo["title"], "coordinates" => coords]),nc),
            "lon" => lonsvar,
            "lat" => latsvar,
            "area" => areasvar,
            "land" => landsvar,
            "thic" => thiccvar
])

writeNetCDFtiles(flds,savenamebase,README)

Third component : `WVELMASS`

In [19]:
# Get the filenames for our first dataset and create BinData struct
dataset = "trsp_3d_set1"
fldname = "WVELMASS"
fnames = nativedir*'/'.*filter(x -> (occursin(".data",x) && occursin(dataset,x)), readdir(nativedir))
savepath = joinpath(writedir,fldname)
if ~ispath(savepath); mkpath(savepath); end
savenamebase = joinpath.(Ref(savepath),fldname)
diaginfo = readAvailDiagnosticsLog(availdiagsfile,fldname)

# Define field- BinData contains the filenames where the data sits so it's only loaded when needed
flddata = BinData(fnames,prec,(n1,n2,n3))
dims = [icvar, jcvar, dep_lvar, timevar]
tilfld = TileData(flddata,tilesize,mygrid)
coords = join(replace([dim.name for dim in dims],"i_c" => "lon", "j_c" => "lat")," ")
flds = Dict([fldname => NCvar(fldname,diaginfo["units"],dims,tilfld,Dict(["long_name" => diaginfo["title"], "coordinates" => coords]),nc),
            "lon" => loncvar,
            "lat" => latcvar,
            "area" => areacvar,
            "land" => land3Dvar,
            "thic" => thiclvar
])

writeNetCDFtiles(flds,savenamebase,README)